In [ ]:
!pip install faker 

In [ ]:
!pip install tqdm

In [3]:
# Gera dados aleatórios 
import faker
import os
import sys
import time
from tqdm import tqdm
import random

# Cria um objeto Faker
fake = faker.Faker()

# Função para gerar os dados de teste
def generate_test_data(num_rows):
  """
  Gera dados de teste e os escreve em um arquivo.

  Argumentos:
    num_rows: Número de linhas a serem geradas.

  Retorno:
    None.
  """
  with open("data/measurements1.txt", "w", encoding="utf-8") as file:
    for _ in tqdm(range(num_rows)):
      # Gera dados aleatórios
      station_name = fake.city()
      temperature = round(fake.random_element([random.uniform(-50, 50), random.uniform(50, 100)]), 1)

      # Escreve a linha no arquivo
      file.write(f"{station_name};{temperature}\n")

# Solicita o número de linhas ao usuário
num_rows = int(input("Digite o número de linhas a serem geradas: "))

# Gera os dados de teste
start_time = time.time()
generate_test_data(num_rows)
end_time = time.time()

# Exibe o tempo real de geração
print("Tempo real de geração:", end_time - start_time, "Minutos")

# Exibe o tamanho real do arquivo
file_size = os.path.getsize("data/measurements1.txt")
print("Tamanho real do arquivo:", file_size, "Megabytes")


Digite o número de linhas a serem geradas:  100000000


100%|████████████████████████████████████| 100000000/100000000 [3:19:11<00:00, 8367.48it/s]

Tempo real de geração: 11951.037693500519 Minutos
Tamanho real do arquivo: 1819848669 Megabytes


In [ ]:
import faker
import os
import sys
import time
from tqdm import tqdm
from multiprocessing import Pool

# Cria um objeto Faker
fake = faker.Faker()

# Função para gerar dados de teste
def generate_test_data(num_rows, chunk_size):
  """
  Gera dados de teste e os escreve em um arquivo.

  Argumentos:
    num_rows: Número de linhas a serem geradas.
    chunk_size: Tamanho do bloco de dados a ser gerado por cada thread.

  Retorno:
    None.
  """
  with open("data/measurements.txt", "w", encoding="utf-8") as file:
    for i in tqdm(range(0, num_rows, chunk_size)):
      # Gera um bloco de dados
      data_chunk = []
      for _ in range(chunk_size):
        station_name = fake.city()
        temperature = round(fake.random_element([random.uniform(-50, 50), random.uniform(50, 100)]), 1)
        data_chunk.append(f"{station_name};{temperature}\n")

      # Escreve o bloco de dados no arquivo
      file.writelines(data_chunk)

# Solicita o número de linhas ao usuário
num_rows = int(input("Digite o número de linhas a serem geradas: "))

# Define o tamanho do bloco de dados
chunk_size = 100000  # Ajuste este valor de acordo com a sua memória disponível

# Número de threads
num_workers = os.cpu_count()

# Gera os dados de teste em paralelo
start_time = time.time()
with Pool(num_workers) as pool:
  pool.starmap(generate_test_data, [(num_rows, chunk_size)])
end_time = time.time()

# Exibe o tempo real de geração
print("Tempo real de geração:", end_time - start_time, "segundos")

# Exibe o tamanho real do arquivo
file_size = os.path.getsize("data/measurements.txt")
print("Tamanho real do arquivo:", file_size, "bytes")


In [ ]:
num_workers = os.cpu_count()

In [ ]:
num_workers

In [ ]:
import faker
import os
from tqdm import tqdm
from multiprocessing import Pool

# Cria um objeto Faker
fake = faker.Faker()

# Função para gerar dados de teste
def generate_test_data(args):
    """
    Gera dados de teste e os escreve em um arquivo.

    Argumentos:
      args: Tupla contendo o número total de linhas a serem geradas e o tamanho do bloco de dados.

    Retorno:
      None.
    """
    num_rows, chunk_size = args
    file_path = "./data/measurements1.txt"

    with open(file_path, "a", encoding="utf-8") as file:
        for _ in tqdm(range(num_rows)):
            station_name = fake.city()
            temperature = round(fake.random_element([fake.random_number(digits=2, fix_len=True) for _ in range(2)]), 1)
            file.write(f"{station_name};{temperature}\n")

# Solicita o número de linhas ao usuário
num_rows = int(input("Digite o número de linhas a serem geradas: "))

# Define o tamanho do bloco de dados
chunk_size = 10000  # Ajuste este valor de acordo com a sua memória disponível

# Número de threads
num_workers = os.cpu_count()

# Gera os dados de teste em paralelo
with Pool(num_workers) as pool:
    pool.map(generate_test_data, [(num_rows // num_workers, chunk_size)] * num_workers)

# Exibe o tamanho real do arquivo
file_size = os.path.getsize("./data/measurements1.txt")
print("Tamanho real do arquivo:", file_size, "bytes")


In [ ]:
!nvidia-smi

In [ ]:
!pip install numpy

In [ ]:
!pip install numba

In [ ]:
import faker
import os
import numpy as np
from numba import jit

# Cria um objeto Faker
fake = faker.Faker()

# Função para gerar dados de teste
@jit(nopython=True)
def generate_test_data(num_rows):
    """
    Gera dados de teste.

    Argumentos:
      num_rows: Número total de linhas a serem geradas.

    Retorno:
      Array numpy contendo os dados gerados.
    """
    data = []
    for _ in range(num_rows):
        station_name = fake.city()
        temperature = round(fake.random_element([fake.random_number(digits=2, fix_len=True) for _ in range(2)]), 1)
        data.append((station_name, temperature))
    return np.array(data)

# Solicita o número de linhas ao usuário
num_rows = int(input("Digite o número de linhas a serem geradas: "))

# Gera os dados de teste
data = generate_test_data(num_rows)

# Escreve os dados em um arquivo
file_path = "data/measurements.txt"
with open(file_path, "w", encoding="utf-8") as file:
    for row in data:
        file.write(f"{row[0]};{row[1]}\n")

# Exibe o tamanho real do arquivo
file_size = os.path.getsize("data/measurements.txt")
print("Tamanho real do arquivo:", file_size, "bytes")


In [ ]:
import faker
import os
import numpy as np

# Cria um objeto Faker
fake = faker.Faker()

# Função para gerar dados de teste
def generate_test_data(num_rows, fake):
    """
    Gera dados de teste.

    Argumentos:
      num_rows: Número total de linhas a serem geradas.
      fake: Objeto Faker para gerar dados falsos.

    Retorno:
      Array numpy contendo os dados gerados.
    """
    data = []
    for _ in range(num_rows):
        station_name = fake.city()
        temperature = round(fake.random_element([fake.random_number(digits=2, fix_len=True) for _ in range(2)]), 1)
        data.append((station_name, temperature))
    return np.array(data)

# Solicita o número de linhas ao usuário
num_rows = int(input("Digite o número de linhas a serem geradas: "))

# Gera os dados de teste
data = generate_test_data(num_rows, fake)

# Escreve os dados em um arquivo
file_path = "./data/measurements.txt"
with open(file_path, "w", encoding="utf-8") as file:
    for row in data:
        file.write(f"{row[0]};{row[1]}\n")

# Exibe o tamanho real do arquivo
file_size = os.path.getsize("./data/measurements.txt")
print("Tamanho real do arquivo:", file_size, "bytes")


In [ ]:
import faker
import os
import numpy as np
from numba import jit
from tqdm import tqdm
import time

# Cria um objeto Faker
fake = faker.Faker()

# Função para gerar dados de teste
def generate_test_data(num_rows, fake):
    """
    Gera dados de teste.

    Argumentos:
      num_rows: Número total de linhas a serem geradas.
      fake: Objeto Faker para gerar dados falsos.

    Retorno:
      Array numpy contendo os dados gerados.
    """
    data = []
    for _ in tqdm(range(num_rows), desc="Gerando dados"):
        station_name = fake.city()
        temperature = round(fake.random_element([fake.random_number(digits=2, fix_len=True) for _ in range(2)]), 1)
        data.append((station_name, temperature))
    return np.array(data)

# Solicita o número de linhas ao usuário
num_rows = int(input("Digite o número de linhas a serem geradas: "))

# Inicia a contagem de tempo
start_time = time.time()

# Gera os dados de teste
data = generate_test_data(num_rows, fake)

# Escreve os dados em um arquivo
file_path = "./data/measurements2.txt"
with open(file_path, "w", encoding="utf-8") as file:
    for row in data:
        file.write(f"{row[0]};{row[1]}\n")

# Finaliza a contagem de tempo
end_time = time.time()

# Exibe o tempo real de geração
print("Tempo real de geração:", end_time - start_time, "segundos")

# Exibe o tamanho real do arquivo
file_size = os.path.getsize("./data/measurements2.txt") / (1024 * 1024)  # Convertendo bytes para megabytes
print("Tamanho real do arquivo:", file_size, "Megabytes")


In [ ]:
import faker
import os
import time
from tqdm import tqdm
import cupy as cp

# Cria um objeto Faker
fake = faker.Faker()

# Função para gerar os dados de teste na GPU
def generate_test_data(num_rows):
    """
    Gera dados de teste e os escreve em um arquivo.

    Argumentos:
      num_rows: Número de linhas a serem geradas.

    Retorno:
      None.
    """
    with open("data/measurements.txt", "w", encoding="utf-8") as file:
        for _ in tqdm(range(num_rows)):
            # Gera dados aleatórios na GPU
            station_name = fake.city()
            temperature = round(cp.random.choice(cp.array([-50, 50, 100]), p=[0.4, 0.3, 0.3]), 1)

            # Escreve a linha no arquivo
            file.write(f"{station_name};{temperature}\n")

# Solicita o número de linhas ao usuário
num_rows = int(input("Digite o número de linhas a serem geradas: "))

# Gera os dados de teste na GPU
start_time = time.time()
generate_test_data(num_rows)
end_time = time.time()

# Exibe o tempo real de geração
print("Tempo real de geração:", end_time - start_time, "segundos")

# Exibe o tamanho real do arquivo
file_size = os.path.getsize("data/measurements.txt")
print("Tamanho real do arquivo:", file_size, "Megabytes")


In [ ]:
!pip install cupy

In [2]:
!poetry add pandas

Using version ^2.2.1 for pandas

Updating dependencies
Resolving dependencies... (0.9s)

Package operations: 3 installs, 1 update, 0 removals

  • Updating python-dateutil (2.8.2 -> 2.9.0.post0): Pending...
  • Installing pytz (2024.1): Pending...
  • Installing tzdata (2024.1): Pending...
  • Installing tzdata (2024.1): Installing...
  • Installing tzdata (2024.1): Installing...
  • Installing pytz (2024.1): Installing...
  • Installing tzdata (2024.1): Installing...
  • Installing pytz (2024.1): Installing...
  • Installing tzdata (2024.1): Installing...
  • Updating python-dateutil (2.8.2 -> 2.9.0.post0): Downloading... 0%
  • Installing pytz (2024.1): Installing...
  • Installing tzdata (2024.1): Installing...
  • Installing pytz (2024.1): Installing...
  • Installing tzdata (2024.1): Installing...
  • Updating python-dateutil (2.8.2 -> 2.9.0.post0): Downloading... 100%
  • Installing pytz (2024.1): Installing...
  • Installing tzdata (2024.1): Installing...
  • Installing pytz (20

# Primeiro teste de leitura do arquivo com pandas

In [8]:
import pandas as pd
from multiprocessing import Pool, cpu_count
from tqdm import tqdm  # importa o tqdm para barra de progresso

CONCURRENCY = cpu_count()

total_linhas = 1_000_000_000  # Total de linhas conhecido
chunksize = 100_000_000  # Define o tamanho do chunk
filename = "data/measurements.txt"  # Certifique-se de que este é o caminho correto para o arquivo

def process_chunk(chunk):
    # Agrega os dados dentro do chunk usando Pandas
    aggregated = chunk.groupby('station')['measure'].agg(['min', 'max', 'mean']).reset_index()
    return aggregated

def create_df_with_pandas(filename, total_linhas, chunksize=chunksize):
    total_chunks = total_linhas // chunksize + (1 if total_linhas % chunksize else 0)
    results = []

    with pd.read_csv(filename, sep=';', header=None, names=['station', 'measure'], chunksize=chunksize) as reader:
        # Envolvendo o iterador com tqdm para visualizar o progresso
        with Pool(CONCURRENCY) as pool:
            for chunk in tqdm(reader, total=total_chunks, desc="Processando"):
                # Processa cada chunk em paralelo
                result = pool.apply_async(process_chunk, (chunk,))
                results.append(result)

            results = [result.get() for result in results]

    final_df = pd.concat(results, ignore_index=True)

    final_aggregated_df = final_df.groupby('station').agg({
        'min': 'min',
        'max': 'max',
        'mean': 'mean'
    }).reset_index().sort_values('station')

    return final_aggregated_df

if __name__ == "__main__":
    import time

    print("Iniciando o processamento do arquivo.")
    start_time = time.time()
    df = create_df_with_pandas(filename, total_linhas, chunksize)
    took = time.time() - start_time

    print(df.head())
    print(f"Processing took: {took:.2f} sec")

Iniciando o processamento do arquivo.


Processando:  10%|████▏                                     | 1/10 [00:05<00:50,  5.58s/it]


        station   min   max       mean
0     Aaronberg -49.7  99.9  38.630482
1  Aaronborough -49.7  99.5  36.750459
2    Aaronburgh -49.9  99.9  36.894530
3     Aaronbury -49.9  99.7  36.687045
4  Aaronchester -49.6  99.7  37.749348
Processing took: 12.03 sec


In [ ]:
# Leitura com Polars

In [6]:
!poetry add polars

Using version ^0.20.13 for polars

Updating dependencies
Resolving dependencies... (0.5s)

Package operations: 1 install, 0 updates, 0 removals

  • Installing polars (0.20.13): Pending...
  • Installing polars (0.20.13): Downloading... 0%
  • Installing polars (0.20.13): Downloading... 10%
  • Installing polars (0.20.13): Downloading... 40%
  • Installing polars (0.20.13): Downloading... 60%
  • Installing polars (0.20.13): Downloading... 80%
  • Installing polars (0.20.13): Downloading... 100%
  • Installing polars (0.20.13): Installing...
  • Installing polars (0.20.13)

Writing lock file


In [39]:
import polars as pl

# Created by Koen Vossen, 
# Github: https://github.com/koenvo
# Twitter/x Handle: https://twitter.com/mr_le_fox
# https://x.com/mr_le_fox/status/1741893400947839362?s=20
def create_polars_df():
    pl.Config.set_streaming_chunk_size(4000000)
    return (
        
        pl.scan_csv("data/measurements1.txt", separator=";", has_header=False, new_columns=["station", "measure"], schema={"station": pl.String, "measure": pl.Float64})
        .group_by(by="station")
        .agg(
            max = pl.col("measure").max(),
            min = pl.col("measure").min(),
            mean = pl.col("measure").mean()
        )
        .sort("station")
        .collect(streaming=True)
    )

if __name__ == "__main__":
    import time

    start_time = time.time()
    df = create_polars_df()
    took = time.time() - start_time
    print(df)
    print(f"Polars Took: {took:.2f} sec")

/tmp/ipykernel_7975/1458131399.py:12: DeprecationWarning: named `by` param is deprecated; use positional `*args` instead.
  .group_by(by="station")


shape: (120_282, 4)
┌──────────────┬───────┬───────┬───────────┐
│ station      ┆ max   ┆ min   ┆ mean      │
│ ---          ┆ ---   ┆ ---   ┆ ---       │
│ str          ┆ f64   ┆ f64   ┆ f64       │
╞══════════════╪═══════╪═══════╪═══════════╡
│ Aaronberg    ┆ 100.0 ┆ -50.0 ┆ 36.758464 │
│ Aaronborough ┆ 100.0 ┆ -50.0 ┆ 37.536199 │
│ Aaronburgh   ┆ 100.0 ┆ -50.0 ┆ 37.171432 │
│ Aaronbury    ┆ 100.0 ┆ -50.0 ┆ 38.144949 │
│ Aaronchester ┆ 100.0 ┆ -50.0 ┆ 36.598826 │
│ …            ┆ …     ┆ …     ┆ …         │
│ Zunigastad   ┆ 99.5  ┆ -49.3 ┆ 36.818891 │
│ Zunigaton    ┆ 100.0 ┆ -48.9 ┆ 37.893103 │
│ Zunigatown   ┆ 100.0 ┆ -49.7 ┆ 37.88753  │
│ Zunigaview   ┆ 100.0 ┆ -49.0 ┆ 42.741876 │
│ Zunigaville  ┆ 100.0 ┆ -49.7 ┆ 38.106135 │
└──────────────┴───────┴───────┴───────────┘
Polars Took: 6.43 sec


In [10]:
!poetry add duckdb

Using version ^0.10.0 for duckdb

Updating dependencies
Resolving dependencies... (0.4s)Resolving dependencies... (0.1s)Resolving dependencies... (0.3s)

Package operations: 1 install, 0 updates, 0 removals

  • Installing duckdb (0.10.0): Pending...
  • Installing duckdb (0.10.0): Downloading... 0%
  • Installing duckdb (0.10.0): Downloading... 10%
  • Installing duckdb (0.10.0): Downloading... 40%
  • Installing duckdb (0.10.0): Downloading... 80%
  • Installing duckdb (0.10.0): Downloading... 100%
  • Installing duckdb (0.10.0): Installing...
  • Installing duckdb (0.10.0)

Writing lock file


In [38]:
import duckdb
import time

def create_duckdb():
    duckdb.sql("""
        SELECT station,
            MIN(measure) AS min_temperature,
            CAST(AVG(measure) AS DECIMAL()) AS mean_temperature,
            MAX(measure) AS max_temperature
        FROM read_csv("data/measurements1.txt", AUTO_DETECT=FALSE, sep=';', columns={'station':VARCHAR, 'measure': 'DECIMAL'})
        GROUP BY station
        ORDER BY station
    """).show()

if __name__ == "__main__":
    import time
    start_time = time.time()
    create_duckdb()
    took = time.time() - start_time
    print(f"Duckdb Took: {took:.2f} sec")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌────────────────────┬─────────────────┬──────────────────┬─────────────────┐
│      station       │ min_temperature │ mean_temperature │ max_temperature │
│      varchar       │  decimal(18,3)  │  decimal(18,3)   │  decimal(18,3)  │
├────────────────────┼─────────────────┼──────────────────┼─────────────────┤
│ Aaronberg          │         -50.000 │           36.758 │         100.000 │
│ Aaronborough       │         -50.000 │           37.536 │         100.000 │
│ Aaronburgh         │         -50.000 │           37.171 │         100.000 │
│ Aaronbury          │         -50.000 │           38.145 │         100.000 │
│ Aaronchester       │         -50.000 │           36.599 │         100.000 │
│ Aaronfort          │         -49.700 │           37.784 │         100.000 │
│ Aaronfurt          │         -49.900 │           37.358 │         100.000 │
│ Aaronhaven         │         -50.000 │           35.586 │         100.000 │
│ Aaronland          │         -49.900 │           37.898 │     

In [ ]:
<a id="ancora01"></a>

In [ ]:
<a id="ancora02"></a>

In [ ]:
<a id="ancora03"></a>

In [ ]:
<a id="ancora04"></a>

In [ ]:
<a id="ancora05"></a>

In [ ]:
<a id="ancora06"></a>

In [ ]:
<a id="ancora07"></a>

In [ ]:
<a id="ancora08"></a>

In [ ]:
<a id="ancora09"></a>

In [ ]:
<a id="ancora10"></a>

In [ ]:
<a id="ancora11"></a>

In [ ]:
<a id="ancora12"></a>

In [ ]:
<a id="ancora13"></a>

In [ ]:
<a id="ancora14"></a>

In [ ]:
<a id="ancora15"></a>

In [ ]:
<a id="ancora16"></a>

In [ ]:
<a id="ancora17"></a>

In [ ]:
<a id="ancora18"></a>

In [ ]:
<a id="ancora19"></a>

In [ ]:
<a id="ancora20"></a>